In [1]:
import urllib.request
import datetime
import time

for i in range(1, 29):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time = now.strftime("%d-%m-%Y__%H-%M-%S")
    
    with open(f'df/id_{str(i)}_{date_and_time}.csv', 'wb') as f:
        f.write(text)

print("VHI is downloaded...")


VHI is downloaded...


In [2]:
import os
import pandas as pd

def read_csv_files_to_df(directory):
    data_frames = []  # список для збереження DataFrame з кожного файлу

    # Проходимо по всіх файлах у вказаній директорії
    for filename in os.listdir(directory):
        reg = filename.split("_")[1]
        file_path = os.path.join(directory, filename)

        try:
            df = pd.read_csv(file_path, index_col=False, header=1)
            df.columns = [col.strip().lower().replace("<br>", "") for col in df.columns]
            df = df.replace(to_replace=r'<.*?>', value='', regex=True)
            df["oblast"] = reg
            data_frames.append(df)

        except Exception as e:
            print(f"Не вдалося зчитати файл {file_path}: {e}")

    # Об'єднуємо всі датафрейми в один
    if data_frames:
        combined_df = pd.concat(data_frames, ignore_index=True)
    else:
        combined_df = pd.DataFrame()

    return combined_df

df = read_csv_files_to_df("df")
print(df.head())


Не вдалося зчитати файл df\.ipynb_checkpoints: [Errno 13] Permission denied: 'df\\.ipynb_checkpoints'
   year  week    smn     smt    vci    tci    vhi oblast
0  1982   1.0  0.059  258.24  51.11  48.78  49.95     10
1  1982   2.0  0.063  261.53  55.89  38.20  47.04     10
2  1982   3.0  0.063  263.45  57.30  32.69  44.99     10
3  1982   4.0  0.061  265.10  53.96  28.62  41.29     10
4  1982   5.0  0.058  266.42  46.87  28.57  37.72     10


In [4]:
obl_dict = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька",
    5: "Житомирська", 6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська",
    9: "Київська", 10: "Кіровоградська", 11: "Луганська", 12: "Львівська",
    13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 16: "Рівненська",
    17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська",
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська",
    25: "Республіка Крим"
}

df["oblast"] = df["oblast"].astype(int).map(obl_dict)


In [15]:
#перетворити ствопчик років з рядка на ціле число
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

print(df[(df["year"]==2018)][:10])

      year  week    smn     smt    vci    tci    vhi          oblast
1872  2018   1.0  0.071  260.87  63.46  35.69  49.57  Кіровоградська
1873  2018   2.0  0.070  259.63  64.12  40.79  52.46  Кіровоградська
1874  2018   3.0  0.071  259.85  64.14  45.75  54.95  Кіровоградська
1875  2018   4.0  0.071  258.79  62.36  46.69  54.53  Кіровоградська
1876  2018   5.0  0.065  257.71  53.36  51.54  52.45  Кіровоградська
1877  2018   6.0  0.063  259.61  42.31  57.21  49.76  Кіровоградська
1878  2018   7.0  0.064  261.47  38.31  55.28  46.80  Кіровоградська
1879  2018   8.0  0.068  262.87  38.86  56.20  47.53  Кіровоградська
1880  2018   9.0  0.077  264.33  40.67  57.98  49.32  Кіровоградська
1881  2018  10.0  0.088  266.63  43.68  55.04  49.36  Кіровоградська


In [16]:
import statistics

def vhi_obl_year(obl, year):
    res = df[(df["oblast"] == obl) & (df["year"] == year)]
    res_vhi = list(res["vhi"])

    print(f"vhi для {obl} області на {year} рік склав \n{res_vhi}")

    print("\n\tMIN:", min(res_vhi))
    print("\tMAX:", max(res_vhi))

    print("\n\tAVG:", statistics.mean(res_vhi))
    print("\tMED:", statistics.median(res_vhi))

vhi_obl_year("Запорізька", 2003)


vhi для Запорізька області на 2003 рік склав 
[49.88, 49.62, 49.35, 49.47, 48.44, 47.89, 47.71, 47.38, -1.0, 46.67, 48.57, 52.37, 54.05, 53.09, 49.87, 47.15, 45.66, 45.24, 44.42, 45.53, 47.71, 48.7, 49.55, 49.95, 49.04, 49.17, 48.97, 48.57, 48.56, 48.49, 46.31, 45.01, 43.34, 44.93, 46.2, 46.81, 47.96, 48.19, 48.97, 50.84, 52.19, 53.4, 51.44, 49.14, 46.99, 46.69, 48.37, 50.05, 50.44, 51.18, 54.03, 58.49]

	MIN: -1.0
	MAX: 58.49

	AVG: 47.90461538461538
	MED: 48.57


In [17]:
def vhi_obls_years(obl, min_year, max_year):
    res = df[(df["oblast"].isin(obl)) & (df["year"].between(min_year, max_year+1))]
    res_vhi = list(res["vhi"])

    print(f"vhi для {obl} області на {min_year}-{max_year} має {len(res_vhi)} значень")

    print("\n\tMIN:", min(res_vhi))
    print("\tMAX:", max(res_vhi))

    print("\n\tAVG:", statistics.mean(res_vhi))
    print("\tMED:", statistics.median(res_vhi))

vhi_obls_years(["Запорізька", "Волинська", "Херсонська"], 1990, 2021)


vhi для ['Запорізька', 'Волинська', 'Херсонська'] області на 1990-2021 має 5148 значень

	MIN: -1.0
	MAX: 80.65

	AVG: 47.26319735819736
	MED: 47.9


In [27]:
def drought_years(percent=0):
    draught_years = []
    oblasti = df["oblast"].unique()
    min_obl_num = int(percent * len(oblasti) / 100)

    for year in df["year"].unique():
        draughts_in = 0
        for obl in oblasti:
            res = df[(df["oblast"] == obl) & (df["year"] == year)]
            res_vhi = list(res["vhi"])
            if len(res_vhi) != 0:
                if statistics.mean(res_vhi) < 35:
                    draughts_in += 1

        if draughts_in > min_obl_num:
            draught_years.append(int(year))

    print(f"Було {len(draught_years)} посушливих років")
    print(draught_years)

drought_years(20)


Було 3 посушливих років
[1985, 1986, 1994]
